In [2]:
import pandas as pd
import os
import altair as alt

C:\Users\user\AppData\Local\Temp\ipykernel_51300\2417646257.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Constants 
IMPORT_PATHS = {
    '2024': 'Z:/Commission - Job Cost/2024/2024 Job Cost Summary.xlsx',
    '2023': 'Z:/Commission - Job Cost/2023/2023 Job Cost Summary.xlsx',
    '2022': 'Z:/Commission - Job Cost/2022/2022 Job Cost Summary.xlsx',
    '2021': 'Z:/Commission - Job Cost/2021/2021 Job Cost Summary.xlsx',
    '2020': 'Z:/Commission - Job Cost/2020/2020 Job Cost Summary with Analysis.xlsx',
    '2019': 'Z:/Commission - Job Cost/2019/2019 Job Cost Summary with Analysis.xlsx',
    '2018': 'Z:/Commission - Job Cost/2018/2018 Job Cost Summary with Analysis.xlsx'
}

COLS_TO_KEEP = [
    'City',
    'Designer',
    'Installer',
    'Contract Date',
    'Customer',
    'Product',
    'Net Sale',
    'Direct Costs',
    'Margin',
    'Comm $',
    'Comm %',
    'Over (Under) Par',
    'Addtl Incent'
]

In [4]:
# Variables
update_bool = False

In [5]:
# Functions
def import_dataset(path, export_name):
    raw = pd.read_excel(path, sheet_name='Raw Data', header= 3)

    #df = raw.drop(labels=['Unnamed: 0', 'Notes', 'Unnamed: 17', 'Unnamed: 18'], axis=1)

    df = raw.loc[:, COLS_TO_KEEP]

    df = df.dropna(axis=0, subset='City')

    file_path = 'Data/' + export_name + 'sales.txt'

    df.to_csv(file_path, index=False)


In [6]:
def combine_dataset():
    data_files = []
    for filename in os.listdir('Data/'):
        file = os.path.join('Data/', filename)
        if os.path.isfile(file):
            data_files.append(file)

    df_concat = pd.concat([pd.read_csv(f) for f in data_files ], ignore_index=True)

    df_concat.to_csv('Data/Combined/Combined data.txt', index=False)


In [7]:
if update_bool == True:

    # Import and combine datasets
    for entry in IMPORT_PATHS:
        import_dataset(IMPORT_PATHS[entry], entry)

    combine_dataset()

In [8]:
# Load data and sort by date
data = pd.read_csv('Data/Combined/Combined data.txt')   
data['Contract Date'] = pd.to_datetime(data['Contract Date'])
data.sort_values(by='Contract Date')

# Load data and bin by year
data_year = pd.read_csv('Data/Combined/Combined data.txt')   
data_year['Contract Date'] = pd.to_datetime(data['Contract Date'])
data_year.sort_values(by='Contract Date')

# Create new columns for year month and date

data['year'] = data['Contract Date'].dt.year.astype(str)
data['month'] = data['Contract Date'].dt.month
data['day'] = data['Contract Date'].dt.day

# Create new columns for product Category and Product Type
data['Sub-Product'] = data['Product']
data['Product'] = 'Other'

bath_prod = ['B', 'B-FR']
kitchen_prod = ['K', 'K-B']
PC_prod = ['FP', 'FP-LC', 'IR', 'IR-LC', 'IR-OP', 'IR-SR', 'IR-Sky', 'LC', 'LC-FP', 'LC-IR', 'OP', 'OP-IR', 'PC', 'Perg', 'VP', 'WP']
Sun_prod = ['SCR', 'SCR-WO', 'SR', 'SR-206', 'SR-306', 'SR-406', 'SR-406-Sky', 'SR-Poly', 'SR-Sky', 'SR-VV', 'SR-WO']
win_prod = ['D', 'W', 'W (A)']
other_prod = []

#if data['Sub-Product'] in bath_prod:
    #data['Product'] = 'B'


data['Product'] = data.apply(lambda row: 'B' if row['Sub-Product'] in bath_prod else row['Product'], axis=1)
data['Product'] = data.apply(lambda row: 'K' if row['Sub-Product'] in kitchen_prod else row['Product'], axis=1)
data['Product'] = data.apply(lambda row: 'PC' if row['Sub-Product'] in PC_prod else row['Product'], axis=1)
data['Product'] = data.apply(lambda row: 'SR' if row['Sub-Product'] in Sun_prod else row['Product'], axis=1)
data['Product'] = data.apply(lambda row: 'W' if row['Sub-Product'] in win_prod else row['Product'], axis=1)


# Create new column for profit
data['Profit'] = data['Margin'] - data['Comm %']


data_year['Contract Date'] = data_year['Contract Date'].dt.year

data = data.sort_values(by=['Product'], ascending=True)


In [14]:
# Generate and sort tags for chart elements

# Year selector
year_tags = list(data['year'].unique())
year_tags.sort()
year_tags = [str(element) for element in year_tags]

year_options = year_tags
year_labels = year_tags 

# Product selector
product_tags = list(data['Product'].unique())
product_tags.sort()
product_tags = [str(element) for element in product_tags]

product_options = product_tags
product_labels = product_tags

# Sub-Product selector
sub_product_tags = list(data['Sub-Product'].unique())
sub_product_tags.sort()
sub_product_tags = [str(element) for element in sub_product_tags]

sub_product_options = sub_product_tags
sub_product_labels = sub_product_tags

# Designer selector
designer_tags = list(data['Designer'].unique())
designer_tags.sort()
designer_tags = [str(element) for element in designer_tags]

designer_options = designer_tags
designer_labels = designer_tags 


In [21]:
# Input selections
brush = alt.selection_point(
    fields=['Sub-Product'],
    bind='legend',
    #on= 'mouseover',
    #nearest=True
)

# Chart elements
year_selection = alt.selection_point(
    name='Contract Date',
    fields=['year'],
    bind=alt.binding_select(
        options= [None] + year_options,
        labels= ['All'] + year_labels,
        ),
)

product_selection = alt.selection_point(
    name='Main',
    fields=['Product'],
    bind= alt.binding_select(
        options= [None] + product_options,
        labels= ['All'] + product_labels,
    ),
)

sub_product_selection = alt.selection_point(
    name='Select',
    fields=['Sub-Product'],
    bind= alt.binding_select(
        options= [None] + sub_product_options,
        labels= ['All'] + sub_product_labels,
    ),
)

designer_selection = alt.selection_point(
    name='Select',
    fields=['Designer'],
    bind= alt.binding_select(
        options= [None] + designer_options,
        labels= ['All'] + designer_labels,
    ),
)

# Chart Elements
year = alt.Chart(data).mark_point(filled=True).encode(
    y=alt.Y('sum(Net Sale)'),
    x='Contract Date',
    color= alt.Color('Sub-Product', legend= alt.Legend(columns= 1, symbolLimit= 0, title= 'Sub-Product')),
    tooltip= ['Contract Date',
                alt.Tooltip('Net Sale', format='.2f'),
                'Product',
                'Sub-Product',
                'Designer',
                alt.Tooltip('Margin', format='.2f'),
                alt.Tooltip('Profit', format='.2f'),
                'Over (Under) Par',
                ],
    opacity= alt.condition(sub_product_selection, alt.value(1), alt.value(0.1)),
).add_params(
    year_selection,
    product_selection,
    designer_selection
).transform_filter(
    year_selection & product_selection & designer_selection & sub_product_selection
).properties(
    width= 600,
    height= 600,
    title= 'Net Sale Over Time'
)

hist_product = alt.Chart(data).mark_bar().encode(
    y=alt.Y('sum(Net Sale)', axis= alt.Axis(title=None)),
    x= alt.X('Product', axis= alt.Axis(title=None, labelAngle= -45)),
    color= 'Product',
    tooltip= ['Product',
            'count()',
            alt.Tooltip('sum(Net Sale)', format='.2f'),
            alt.Tooltip('average(Margin)', format='.2f'),
            alt.Tooltip('average(Comm %)', format='.2f'),
            alt.Tooltip('average(Profit)', format='.2f'),
            ],
    opacity= alt.condition(sub_product_selection, alt.value(1), alt.value(0.1))
).add_params(
    year_selection,
    product_selection,
    designer_selection
).transform_filter(
    year_selection & product_selection & designer_selection
).properties(
    width= 100,
    height= 600,
    title= 'Products Stats'
)

hist_sub_product = alt.Chart(data).mark_bar().encode(
    y=alt.Y('sum(Net Sale)', axis= alt.Axis(title=None)),
    x= alt.X('Sub-Product', axis= alt.Axis(title=None, labelAngle= -45)),
    color= 'Sub-Product',
    tooltip= ['Sub-Product',
            'count()',
            alt.Tooltip('sum(Net Sale)', format='.2f'),
            alt.Tooltip('average(Margin)', format='.2f'),
            alt.Tooltip('average(Comm %)', format='.2f'),
            alt.Tooltip('average(Profit)', format='.2f'),
            ],
    opacity= alt.condition(sub_product_selection, alt.value(1), alt.value(0.1))
).add_params(
    year_selection,
    product_selection,
    designer_selection,
    sub_product_selection
).transform_filter(
    year_selection & product_selection & designer_selection
).properties(
    width= 600,
    height= 600,
    title= 'Sub-Product stats'
)

dashboard = (year + year.transform_regression('Contract Date', 'Net Sale').mark_line() | hist_product | hist_sub_product)
dashboard

alt.HConcatChart(...)

In [11]:
# Input selections
brush_2 = alt.selection_point(
    fields=['Product'],
    #bind='legend',
    #on= 'mouseover',
    #nearest=True
)

# Chart elements
year_selection = alt.selection_point(
    name='Contract Date',
    fields=['year'],
    bind=alt.binding_select(
        options= [None] + year_options,
        labels= ['All'] + year_labels,
        ),
)

product_selection = alt.selection_point(
    name='Product',
    fields=['Product'],
    bind= alt.binding_select(
        options= [None] + product_options,
        labels= ['All'] + product_labels,
    ),
)

designer_selection = alt.selection_point(
    name='Designer',
    fields=['Designer'],
    bind= alt.binding_select(
        options= [None] + designer_options,
        labels= ['All'] + designer_labels,
    ),
)

# Chart Elements
margin = alt.Chart(data).mark_point(filled=True).encode(
    y= alt.Y('Margin'),
    x= alt.X('Comm %'),
    color= alt.Color('Product', legend= alt.Legend(columns= 1, symbolLimit=0)),
    tooltip= ['Customer',
                'Contract Date',
                'Product',
                'Designer',
                alt.Tooltip('sum(Net Sale)', format='.2f'),
                alt.Tooltip('Margin', format='.2f'),
                alt.Tooltip('Comm %', format='.2f'),
                alt.Tooltip('Profit', format='.2f'),
                ],
    #opacity= alt.condition(brush_2, alt.value(1), alt.value(0.1))
).add_params(
    year_selection,
    product_selection,
    designer_selection
).transform_filter(
    year_selection & product_selection & designer_selection
).properties(
    width= 1600,
    height= 600,
    title= 'Projects sold'
)

# Regression
margin_chart = margin + margin.transform_regression('Comm %', 'Margin').mark_line()
margin_chart

alt.LayerChart(...)

In [12]:
dashboard.save('index.html', embed_options={'renderer': 'svg'})